# Python Client for GCS

Google Cloud Storage (GCS), buckets to hold all your information, data, files.

This notebook illustrates some common interactions with GCS and provides tips on using the Python Client to:
- List buckets
- List files
- Download files
- Upload files
- more

Resources:
- [Product](https://cloud.google.com/storage)
- [Client API](https://github.com/googleapis/python-storage)
- [Client API Documentation](https://cloud.google.com/python/docs/reference/storage/latest)


---
## Setup

inputs:

In [2]:
EXPERIMENT = 'gcs'

packages:

In [3]:
from google.cloud import storage

clients:

In [4]:
gcs = storage.Client()

parameters:

In [5]:
DIR = f'temp/{EXPERIMENT}'

environment:

In [6]:
!rm -rf {DIR}
!mkdir -p {DIR}

---
## Buckets

## Files (blobs)

### Downloading

### Uploading